In [2]:
from langchain_community.document_loaders import WebBaseLoader

In [5]:
loader = WebBaseLoader("https://www.lequipe.fr/Rugby/Actualites/Ecosse-trois-changements-face-a-la-france-pour-le-tournoi-des-six-nations/1447566")
data = loader.load()

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1000,
    chunk_overlap=20,
)

texts = text_splitter.split_documents(data)

In [23]:
print("Preview:")
print(texts[0].page_content, "\n")



Preview:
Écosse : trois changements face à la France pour le Tournoi des Six Nations - L'Équipe 



In [32]:
from vertexai.language_models import TextGenerationModel

generation_model = TextGenerationModel.from_pretrained("text-bison@001")

In [34]:
prompt = "generate hashtag:" + texts[0].page_content

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)





#Écosse #France #TournoiDesSixNations #LÉquipe


In [13]:
# Utils
from langchain.schema import HumanMessage, SystemMessage
from langchain.llms import VertexAI
from langchain.embeddings import VertexAIEmbeddings
from langchain.chat_models import ChatVertexAI
from google.cloud import aiplatform
import time
from typing import List

# LangChain
import langchain
from pydantic import BaseModel

print(f"LangChain version: {langchain.__version__}")

# Vertex AI

print(f"Vertex AI SDK version: {aiplatform.__version__}")
#Next you will define some utility functions that you will use for the Vertex AI Embeddings API

# Utility functions for Embeddings API with rate limiting
def rate_limit(max_per_minute):
    period = 60 / max_per_minute
    print("Waiting")
    while True:
        before = time.time()
        yield
        after = time.time()
        elapsed = after - before
        sleep_time = max(0, period - elapsed)
        if sleep_time > 0:
            print(".", end="")
            time.sleep(sleep_time)




LangChain version: 0.1.4
Vertex AI SDK version: 1.40.0


In [21]:
from langchain_core.embeddings import Embeddings
from langchain_community.llms.vertexai import _VertexAICommon


class CustomVertexAIEmbeddings(_VertexAICommon, Embeddings):
    requests_per_minute: int
    num_instances_per_batch: int

    # Overriding embed_documents method
    def embed_documents(self, texts: List[str]):
        limiter = rate_limit(self.requests_per_minute)
        results = []
        docs = list(texts)

        while docs:
            # Working in batches because the API accepts maximum 5
            # documents per request to get embeddings
            head, docs = (
                docs[: self.num_instances_per_batch],
                docs[self.num_instances_per_batch :],
            )
            chunk = self.client.get_embeddings(head)
            results.extend(chunk)
            next(limiter)

        return [r.values for r in results]
    def embed_query(self, text: str) -> List[float]:
            """Embed a text.

            Args:
                text: The text to embed.

            Returns:
                Embedding for the text.
            """
            embeddings = self.embed([text], 1, "RETRIEVAL_QUERY")
            return embeddings[0]    

# LLM model
llm = VertexAI(
    model_name="text-bison@001",
    max_output_tokens=256,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)


# Embedding
EMBEDDING_QPM = 100
EMBEDDING_NUM_BATCH = 5
embeddings = CustomVertexAIEmbeddings(
     model_name="textembedding-gecko@002",
    requests_per_minute=EMBEDDING_QPM,
    num_instances_per_batch=EMBEDDING_NUM_BATCH,
)


In [22]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS

# Embed your texts
db = FAISS.from_documents(texts, embeddings)

AttributeError: 'NoneType' object has no attribute 'get_embeddings'